In [198]:
import RNA
import numpy as np

from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()

import subprocess
import matplotlib.pyplot as plt
from PIL import Image
from IPython.display import SVG, display
from collections import Counter
from collections import defaultdict

import difflib
import sys
import os
import random
import string
import time
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

# from helper import print_moves

import pandas as pd

sys.path.append('../')
from pretty_print_path import print_moves
import findpath_librna
import findpath


In [284]:
input_file = "dataset_104_train.csv"
input_file2 = "dataset_104_vec_train.csv"

dataframe = pd.read_csv(input_file, index_col=0)
# dataframe['target'] = np.where(dataframe["3"]==1, 1, 0)
dataframe = dataframe.drop(labels="s", axis=1)
dataframe = dataframe.drop(labels="i", axis=1)
dataframe = dataframe.drop(labels="j", axis=1)
# dataframe = dataframe.drop(labels="found", axis=1)

# dataframe

vec_dataframe = pd.read_csv(input_file2, index_col=0)
# vec_dataframe
dataframe = pd.concat([dataframe, vec_dataframe], axis=1)

dataframe

,target,ijd,thisclose,lastclose,cd,en_pos,i_shift,j_shift,insert_or_delete,0,...,10,11,12,13,14,15,16,17,18,19
0,1,0.363636,0.285714,0.142857,1,0.000000,0,0,0,0.379594,...,6.618096e-04,3.768748e-02,3.131309e-01,0.379594,0.067139,0.001733,0.000007,3.583778e-09,2.872489e-13,3.359235e-18
1,1,0.181818,0.333333,0.142857,1,0.034483,0,1,0,0.379594,...,6.618096e-04,3.768748e-02,3.131309e-01,0.379594,0.067139,0.001733,0.000007,3.583778e-09,2.872489e-13,3.359235e-18
2,0,0.818182,0.238095,0.142857,1,0.034483,0,0,0,0.379594,...,6.618096e-04,3.768748e-02,3.131309e-01,0.379594,0.067139,0.001733,0.000007,3.583778e-09,2.872489e-13,3.359235e-18
3,0,0.954545,0.047619,0.142857,1,0.103448,0,0,0,0.379594,...,6.618096e-04,3.768748e-02,3.131309e-01,0.379594,0.067139,0.001733,0.000007,3.583778e-09,2.872489e-13,3.359235e-18
4,1,0.636364,0.333333,0.142857,1,0.206897,0,0,0,0.379594,...,6.618096e-04,3.768748e-02,3.131309e-01,0.379594,0.067139,0.001733,0.000007,3.583778e-09,2.872489e-13,3.359235e-18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4926,1,0.166667,1.000000,0.000000,0,1.000000,1,0,1,0.513713,...,7.875288e-28,2.585954e-14,7.441132e-09,0.000005,0.000524,0.009191,0.028921,4.183780e-02,8.569017e-02,1.100865e-01
4927,1,0.200000,1.000000,1.000000,0,0.000000,0,0,1,0.168993,...,1.464808e-21,8.502979e-15,2.446749e-09,0.000002,0.000172,0.003022,0.009516,1.548947e-02,9.531556e-02,4.157920e-01
4928,1,0.200000,1.000000,1.000000,1,0.663043,0,1,1,0.168993,...,1.464808e-21,8.502979e-15,2.446749e-09,0.000002,0.000172,0.003022,0.009516,1.548947e-02,9.531556e-02,4.157920e-01
4929,1,0.400000,1.000000,1.000000,0,0.717391,0,1,1,0.168993,...,1.464808e-21,8.502979e-15,2.446749e-09,0.000002,0.000172,0.003022,0.009516,1.548947e-02,9.531556e-02,4.157920e-01


In [285]:
train, test = train_test_split(dataframe, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

3155 train examples
789 validation examples
987 test examples


In [286]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('target')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

batch_size = 5 # A small batch sized is used for demonstration purposes
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

# ## Understand the input pipeline
# # Now that we have created the input pipeline, let's call it to see the format of the data it returns. We have used a small batch size to keep the output readable.

# for feature_batch, label_batch in train_ds.take(1):
#   print('Every feature:', list(feature_batch.keys()))
#   print('A batch of ages:', feature_batch['4'])
#   print('A batch of targets:', label_batch )


In [287]:
# We will use this batch to demonstrate several types of feature columns
example_batch = next(iter(train_ds))[0]

# A utility method to create a feature column
# and to transform a batch of data
def demo(feature_column):
  feature_layer = layers.DenseFeatures(feature_column)
  print(feature_layer(example_batch).numpy())

# # numeric columns
photo_count = feature_column.numeric_column('4')
demo(photo_count)
photo_count2 = feature_column.numeric_column('5')

crossed_feature = feature_column.crossed_column(['4', '1'], hash_bucket_size=10)
demo(feature_column.indicator_column(crossed_feature))

# # column 7 is a catigorical column
# animal_type = feature_column.categorical_column_with_vocabulary_list(
#       '7', [0, 1])
# animal_type_one_hot = feature_column.indicator_column(animal_type)
# demo(animal_type_one_hot)

[[3.1255597e-01]
 [2.9405925e-04]
 [4.9473299e-03]
 [5.6313086e-01]
 [4.0208951e-01]]
[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]


## Choose which columns to use
We have seen how to use several types of feature columns. Now we will use them to train a model. The goal of this tutorial is to show you the complete code (e.g. mechanics) needed to work with feature columns. We have selected a few columns to train our model below arbitrarily.

Key point: If your aim is to build an accurate model, try a larger dataset of your own, and think carefully about which features are the most meaningful to include, and how they should be represented.

In [288]:
feature_columns = []

# ijd	thisclose	lastclose	cd	en_pos

# numeric cols
for header in ['ijd',	'thisclose',	'lastclose',	'en_pos', 'i_shift',	'j_shift',	'insert_or_delete']:
  feature_columns.append(feature_column.numeric_column(header))

# numeric cols
for header in ['0',	'1',	'2',	'3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19']:
  feature_columns.append(feature_column.numeric_column(header))

# categoriacal indicator_columns
indicator_column_names = ['cd']
for col_name in indicator_column_names:
  categorical_column = feature_column.categorical_column_with_vocabulary_list(
      col_name, dataframe[col_name].unique())
  indicator_column = feature_column.indicator_column(categorical_column)
  feature_columns.append(indicator_column)

In [217]:
# only use lowest energy as indicator

feature_columns = []

# numeric cols
for header in ['en_pos']:
  feature_columns.append(feature_column.numeric_column(header))

In [289]:
feature_columns

[NumericColumn(key='ijd', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='thisclose', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='lastclose', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='en_pos', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='i_shift', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='j_shift', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='insert_or_delete', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='0', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='1', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='2', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),


### Create a feature layer
Now that we have defined our feature columns, we will use a [DenseFeatures](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/DenseFeatures) layer to input them to our Keras model.

Earlier, we used a small batch size to demonstrate how feature columns worked. We create a new input pipeline with a larger batch size.

In [290]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [291]:
epochs = 50

model = tf.keras.Sequential([
  feature_layer,
  # layers.Dense(128, activation='relu'),
  # layers.Dense(128, activation='relu'),
  layers.Dense(256, activation='relu'),
  layers.Dense(256, activation='relu'),
  layers.Dropout(.1),
  layers.Dense(1)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(train_ds,
          validation_data=val_ds,
          epochs=epochs)

Epoch 1/50
Consider rewriting this model with the Functional API.


Consider rewriting this model with the Functional API.


Consider rewriting this model with the Functional API.


Consider rewriting this model with the Functional API.


77/99 [======================>.......] - ETA: 0s - loss: 0.5481 - accuracy: 0.6928WARNING:tensorflow:Layers in a Sequential model should only have a single input tensor, but we receive a <class 'dict'> input: {'ijd': <tf.Tensor 'ExpandDims_23:0' shape=(None, 1) dtype=float64>, 'thisclose': <tf.Tensor 'ExpandDims_27:0' shape=(None, 1) dtype=float64>, 'lastclose': <tf.Tensor 'ExpandDims_26:0' shape=(None, 1) dtype=float64>, 'cd': <tf.Tensor 'ExpandDims_20:0' shape=(None, 1) dtype=int64>, 'en_pos': <tf.Tensor 'ExpandDims_21:0' shape=(None, 1) dtype=float64>, 'i_shift': <tf.Tensor 'ExpandDims_22:0' shape=(None, 1) dtype=int64>, 'j_shift': <tf.Tensor 'ExpandDims_25:0' shape=(None, 1) dtype=int64>, 'insert_or_delete': <tf.Tensor 'ExpandDims_24:0' shape=(None, 1) dtype=int64>, '0': <tf.Tensor 'ExpandDims:0' shape=(None, 1) dtype=float64>, '1': <tf.Tensor 'ExpandDims_1:0' shape=(None, 1) dtype=float64>, '2': <tf.Tensor 'ExpandDims_12:0' shape=(None, 1) dtype=float64>, '3': <tf.Tensor 'ExpandDi

Consider rewriting this model with the Functional API.


99/99 [==============================] - 2s 6ms/step - loss: 0.5398 - accuracy: 0.7036 - val_loss: 0.5220 - val_accuracy: 0.7224
Epoch 2/50
99/99 [==============================] - 0s 2ms/step - loss: 0.4851 - accuracy: 0.7569 - val_loss: 0.4943 - val_accuracy: 0.7579
Epoch 3/50
99/99 [==============================] - 0s 2ms/step - loss: 0.4462 - accuracy: 0.7803 - val_loss: 0.4583 - val_accuracy: 0.7643
Epoch 4/50
99/99 [==============================] - 0s 2ms/step - loss: 0.4127 - accuracy: 0.7981 - val_loss: 0.4376 - val_accuracy: 0.7630
Epoch 5/50
99/99 [==============================] - 0s 3ms/step - loss: 0.3927 - accuracy: 0.8079 - val_loss: 0.4334 - val_accuracy: 0.7693
Epoch 6/50
99/99 [==============================] - 0s 2ms/step - loss: 0.3605 - accuracy: 0.8273 - val_loss: 0.4024 - val_accuracy: 0.8010
Epoch 7/50
99/99 [==============================] - 0s 2ms/step - loss: 0.3367 - accuracy: 0.8466 - val_loss: 0.4055 - val_accuracy: 0.7909
Epoch 8/50
99/99 [=============

In [293]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

31/31 [==============================] - 0s 2ms/step - loss: 0.5097 - accuracy: 0.8764
Accuracy 0.8763931393623352


In [282]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

31/31 [==============================] - 0s 2ms/step - loss: 0.4810 - accuracy: 0.8480
Accuracy 0.848024308681488


In [222]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

31/31 [==============================] - 0s 964us/step - loss: 0.5752 - accuracy: 0.7214
Accuracy 0.7213779091835022


In [ ]:
# 31/31 [==============================] - 0s 1ms/step - loss: 0.6488 - accuracy: 0.7335
# Accuracy 0.7335359454154968

# 31/31 [==============================] - 0s 862us/step - loss: 0.5798 - accuracy: 0.6950
# Accuracy 0.695035457611084

In [13]:
# Save the entire model as a SavedModel.
# !mkdir -p saved_model
# model.save('saved_model/my_model')

# from tensorflow.keras.models import Sequential, save_model, load_model

# filepath = './saved_model'
# save_model(model, filepath)

# Morgan & Higgs: Findpath with SW 1

In [107]:
input_file = "dataset_100.csv"
df = pd.read_csv(input_file)

sequence, s1, s2 = df.loc[0]


search_width_multiplier = 4
fp = findpath.findpath_single(sequence, s1, s2, search_width_multiplier=search_width_multiplier, mp=True)
result = fp.get_en()/100.0
path = fp.get_path()

s = s1
pt2 = list(RNA.ptable(s2))
fc = RNA.fold_compound(sequence)


def find_moves(s_ptable, t_ptable):
    """
    generator function, yields possible structures 1 move away
    from the original structure by finding fitting i and j with
    RNA pair and loop tables
    s_ptable: current ptable
    t_ptable: s2 end ptable
    """
    # loop table
    ls = RNA.loopidx_from_ptable(s_ptable)
    for i in range(len(s_ptable)):
        if i == 0:
            continue
        if s_ptable[i] == 0 and t_ptable[i] > i:
            j = t_ptable[i]
            # found j has to be empty and currently on the same loop as i
            if s_ptable[j] == 0 and ls[i] == ls[j]:
                yield i, j
        # test for bp removal: i has to be paired with a different j in s2
        j = s_ptable[i]
        # dont remove things which are present in s2
        if s_ptable[i] > i and s_ptable[i] != s_ptable[j] and\
                s_ptable[i] != t_ptable[i] and s_ptable[j] != t_ptable[j]:
            yield -i, -j


def fp_call(sequence, s1, s2, search_width_multiplier = 20):    
    fp = findpath.findpath_single(sequence, s1, s2, search_width_multiplier=search_width_multiplier, mp=True)
    result = fp.get_en()/100.0
    path = fp.get_path()
    # return result, path
    return result, path


def ij_distance(last_move, this_move, ij_moves):
    # how far is the last move away from the current move.
    # it is likely, that the next move is close to the last one
    # there are better distance metrices probably...

    # ij move list is supposed to be sorted, find indices
    
    ijmoves = ij_moves + [last_move]
    ijmoves.sort(key=lambda x: (x[0], x[1]))

    pos_old = ijmoves.index(last_move)
    pos_new = ijmoves.index(this_move)

    distance = abs(pos_old-pos_new)/len(ijmoves)
    
    
    # moves left in vicinity out of total moves
    
    thisi, thisj = this_move
    lasti, lastj = last_move
    thisclose = 0
    lastclose = 0

    for i, j in ij_moves:
        if (abs(i-thisi) < 5) and (abs(j-thisj) < 5):
            thisclose += 1
        if (abs(i-lasti) < 5) and (abs(j-lastj) < 5):
            lastclose += 1
    

    thisclose /= len(ij_moves)
    lastclose /= len(ij_moves)

    # print ("thisclose", thisclose, lastclose, distance)
    return distance, thisclose, lastclose

    print (distance)

# sample call
ij_moves = [(3, 62), (4, 61), (6, 60), (7, 59), (9, 57), (10, 56), (11, 55), (12, 54), (15, 52), (16, 51), (17, 50), (18, 39), (19, 38), (20, 36), (21, 35), (22, 34), (23, 33), (24, 32), (42, 49), (43, 48), (64, 99)]
last_move = (2, 63)
this_move = (6, 60)
ij_distance(last_move, this_move, ij_moves)

# 

def config_distance(pt, move):
    """
    are we extending / removing the outside/inside layer of a loop or adding something in the middle?
    """
    i = move[0]
    j = move[1]
    points = 0

    # if we're extending from outside to inside, the position i+1 and j-1 should be ideally unpaired
    # inside to outside: i-1 and j+1 should be ideally unpaired

    if i>0:
        # print ("add") 
        # outside/inside paired?        
        if j+1 < pt[0] and i-1 > 0: # outside - boundary check
            if pt[i-1] == j+1:
                points += 1
        if pt[i+1] == j-1:
            points += 1
    if i<0:
        # print ("del")
        i, j = -i, -j
        # outside/inside paired?
        if j+1 < pt[0] and i-1 > 0: # outside - boundary check
            if pt[i-1] == j+1:
                points += 1
        if pt[i+1] == j-1:
            points += 1
        elif pt[i+1] == 0 and pt[j-1] == 0:
            pass

    if points == 2:
        points = 0
    return points


s = s1
lasts = s
lasti = None
lastj = None

for e, (a,b, en) in enumerate(path):
    if (a,b) == (0,0):
        continue  

    # check where we can go, compare with our best move. 
    pt = list(RNA.ptable(s))

    # check available moves, save them, sort them    
    avail_moves = []
    ij_moves = []
    found_pos = None

    for pos, (i,j) in enumerate(find_moves(pt, pt2)):    
        next_en = fc.eval_move_pt(pt, i, j)
        # mark where we found our move
        found = (i,j) == (a,b)
        avail_moves.append((i, j, next_en, found))
        ij_moves.append((abs(i),abs(j)))

    # sort moves independent of delete insert moves

    ij_moves.sort(key=lambda x: (x[0], x[1]))


    avail_moves.sort(key=lambda x: x[2])
    found_list = [x[3] for x in avail_moves]
    en_list = np.array([[x[2] for x in avail_moves]])
    en_list_scaled = min_max_scaler.fit_transform(en_list.T).T[0]
    

    # find where our move is after sorting
    found_pos = found_list.index(True)
    rel_pos = found_pos * 1.0 / len(found_list)

    # print (e, a,b, 'found at pos:', found_pos, 'of', len(avail_moves), ':',  1-rel_pos)
    # print (avail_moves, a, b)

    # if s != lasts:
        # print ("---") 
        # print (s)
        # print ("---") 

    # for every move we take we have to run a new findpath, see if this move will yield the ideal result
    
    for pos, (i,j, en, found) in enumerate(avail_moves):
        if i > 0:
            snew = s[:i-1] + "(" + s[i:j-1] + ")" + s[j:]
        if i < 0:
            snew = s[:-i-1] + "." + s[-i:-j-1] + "." + s[-j:]
        ptnew = list(RNA.ptable(snew))

        result_new, path = fp_call(sequence, snew, s2)

        if result_new <= result:
            pos_result = 1
        else:
            pos_result = 0

        if found: found = "<-- taken"
        else: found = ""

        this_move = (abs(i), abs(j))
        last_move = (lasti, lastj)

        if lasti:
            # print (this_move, last_move, ij_moves)
            ijd, thisclose, lastclose = ij_distance(last_move, this_move, ij_moves)
        else:
            ijd, thisclose, lastclose = 0, 0, 0

        cd = config_distance(pt, this_move)

        if lasti:
            print (f' {snew[0:20]}, {i}, {j}, {result_new}/{pos_result}: {ijd:2.2f}, {thisclose:2.2f}, {lastclose:2.2f}, {cd}, {en}, {en_list_scaled[pos]:2.2f} {found}')

            sample = 1, ijd, thisclose, lastclose, cd, en_list_scaled[pos], 1
            sample_test_ds = sample_to_dataset(sample, test)
            result_predictor = model.predict(sample_test_ds)[0][0]

            print (f' {1}, {ijd:2.2f}, {thisclose:2.2f}, {lastclose:2.2f}, {cd}, {en_list_scaled[pos]:2.2f}, 1: prediction: {result_predictor}')



    # if e==63:
    #     print (avail_moves)

    # update s for the next iteration
    
    lasts = s
    lasti = abs(a)
    lastj = abs(b)
    if a > 0:
        s = s[:a-1] + "(" + s[a:b-1] + ")" + s[b:]
    if a < 0:
        s = s[:-a-1] + "." + s[-a:-b-1] + "." + s[-b:]

    if e>1: 
        break



 ..((.((.(((...((((((, -12, -54, -14.2/1: 0.36, 0.29, 0.14, 1, 90, 0.00 <-- taken
 1, 0.36, 0.29, 0.14, 1, 0.00, 1: prediction: 0.6828221082687378
 ..((.(..((((..((((((, -7, -59, -14.2/1: 0.18, 0.33, 0.14, 1, 110, 0.03 
 1, 0.18, 0.33, 0.14, 1, 0.03, 1: prediction: -0.31268346309661865
 ..((.((.((((..((((((, -24, -32, -13.3/0: 0.82, 0.24, 0.14, 1, 110, 0.03 
 1, 0.82, 0.24, 0.14, 1, 0.03, 1: prediction: -2.4331817626953125
 ..((.((.((((..((((((, -64, -99, -12.7/0: 0.95, 0.05, 0.14, 1, 150, 0.10 
 1, 0.95, 0.05, 0.14, 1, 0.10, 1: prediction: -8.228426933288574
 ..((.((.((((..(((((., -20, -36, -14.2/1: 0.64, 0.33, 0.14, 1, 210, 0.21 
 1, 0.64, 0.33, 0.14, 1, 0.21, 1: prediction: -1.486687183380127
 ..((.((.((((..((((.(, -19, -38, -14.2/1: 0.59, 0.24, 0.14, 1, 300, 0.36 
 1, 0.59, 0.24, 0.14, 1, 0.36, 1: prediction: -1.3077186346054077
 ..((.((..(((..((((((, -9, -57, -14.2/1: 0.23, 0.29, 0.14, 1, 310, 0.38 
 1, 0.23, 0.29, 0.14, 1, 0.38, 1: prediction: -0.09669250249862671
 ..((.((.((((..

In [16]:
# A few random samples


# Generate predictions for samples
predictions = model.predict(test_ds)

predictions
# sample_test
# test_ds

array([[-1.3807142],
       [-0.6300998],
       [-2.0548708],
       ...,
       [-1.4828502],
       [-1.810354 ],
       [-1.9349219]], dtype=float32)

In [105]:
# A utility method to create a tf.data from a dictionary
def dict_to_dataset(inputdict, labels, shuffle=True, batch_size=32):
  # dataframe = dataframe.copy()
  # labels = dataframe.pop('target')
  ds = tf.data.Dataset.from_tensor_slices((inputdict, labels))
  # ds = tf.data.Dataset.from_tensor_slices([45240,  0.5,  0.142857,  0.047619,  0,  0.930233,       0])
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))  
  print ('ds', ds)
  ds = ds.batch(batch_size)
  return ds

def sample_to_dataset(sample, test):

  sample_test = test.iloc[0:1].copy()
  row0 = sample_test.index[0]
  sample_test.at[row0 ,'Unnamed: 0'] = 2 # irrelevant
  sample_test.at[row0 ,'4'] = sample[1]
  sample_test.at[row0 ,'5'] = sample[2]
  sample_test.at[row0 ,'6'] = sample[3]
  sample_test.at[row0 ,'7'] = sample[4]
  sample_test.at[row0 ,'8'] = sample[5]
  sample_test.at[row0 ,'target'] = 2 # irrelevant

  sample_test_ds = df_to_dataset(sample_test, shuffle=False, batch_size=batch_size)

  return sample_test_ds



sample = [1, 0.82, 0.44, 0.24, 1, 0.05, 1]
sample = 1, 0.36, 0.29, 0.14, 1, 0.00, 1
sample_test_ds = sample_to_dataset(sample, test)
result_predictor = model.predict(sample_test_ds)[0][0]
print (result_predictor)


predictions = model.predict(sample_test_ds)
predictions

0.7363248


array([[0.7363248]], dtype=float32)

In [192]:
filename_samples = f'./dataset_100.csv'
samples_df = pd.read_csv(filename_samples)

sequence = ''
s1 = ''
s2 = ''

for index, row in samples_df.iterrows():
    if index != 56:
        continue
    sequence = row.sequence
    s1 = row.s1
    s2 = row.s2


print (sequence)
print (s1)
print (s2)

fc = RNA.fold_compound(sequence)
en1 = fc.eval_structure(s1)
en2 = fc.eval_structure(s2)

def fp_call_sw(sequence, s1, s2, search_width):    
    fp = findpath.findpath_single(sequence, s1, s2, search_width=search_width, mp=True)
    result = fp.get_en()/100.0
    path = fp.get_path()
    # return result, path
    return result, path


sw = 1
sw = 100

r, p = fp_call_sw(sequence, s1, s2, sw)
p = [(i[0], i[1]) for i in p]

print_moves(sequence, s1, s2, p)


GGCUCAGUCGACGCUGCACCGCCUAGACCCGAUCGCUAAGAAGGAAAACAGUCAGAUGACCGAGUAUUCGAACGCUGCCGUAUCUCUCGCAUAUAACACA
(((.((((....))))....))).....((..((.....)).))......(((....)))((((....((........)).....))))...........
((..((((....))))..))..............((..(((.((...((.(.(((.((..(((....)))..))))).))).))))).))..........
GGCUCAGUCGACGCUGCACCGCCUAGACCCGAUCGCUAAGAAGGAAAACAGUCAGAUGACCGAGUAUUCGAACGCUGCCGUAUCUCUCGCAUAUAACACA
(((.((((....))))....))).....((..((.....)).))......(((....)))((((....((........)).....))))........... [   0,    0 ] -12.90
.((.((((....))))....))......((..((.....)).))......(((....)))((((....((........)).....))))........... [  -1,  -23 ]  -9.40
..(.((((....))))....).......((..((.....)).))......(((....)))((((....((........)).....))))........... [  -2,  -22 ]  -6.10
....((((....))))............((..((.....)).))......(((....)))((((....((........)).....))))........... [  -3,  -21 ]  -9.30
(...((((....))))...)........((..((.....)).))......(((....)))((((....((........)).....))))........... [   1, 

-4.8

In [194]:
s = s1
fc = RNA.fold_compound(sequence)
pt = list(RNA.ptable(s))
pt2 = list(RNA.ptable(s2))

lasts = s
lasti = 0
lastj = 0
last_move = (0,0)

en_max = fc.eval_structure(s)

found_moves = [(0, 0)]

while True:

    # check where we can go, compare with our best move. 
    pt = list(RNA.ptable(s))

    # check available moves, save them, sort them    
    avail_moves = []
    ij_moves = []
    found_pos = None

    # collect moves from the generator function, then sort them.
    for pos, (i,j) in enumerate(find_moves(pt, pt2)):    
        next_en = fc.eval_move_pt(pt, i, j)
        # mark where we found our move
        found = (i,j) == (a,b)
        avail_moves.append((i, j, next_en, found))
        ij_moves.append((abs(i),abs(j)))


    avail_moves.sort(key=lambda x: x[2])
    found_list = [x[3] for x in avail_moves]
    en_list = np.array([[x[2] for x in avail_moves]])
    en_list_scaled = min_max_scaler.fit_transform(en_list.T).T[0]


    # print (s, pt, avail_moves)
    # print (en_list_scaled)

    candidates = []

    for pos, (i,j, en, found) in enumerate(avail_moves):

        this_move = (abs(i), abs(j))
        last_move = (lasti, lastj)

        if lasti:
            # print (this_move, last_move, ij_moves)
            ijd, thisclose, lastclose = ij_distance(last_move, this_move, ij_moves)
        else:
            ijd, thisclose, lastclose = 0, 0, 0

        cd = config_distance(pt, this_move)

        en_scaled = en_list_scaled[pos]

        # run the prediction
        sample = 1, ijd, thisclose, lastclose, cd, en_list_scaled[pos], 1
        sample_test_ds = sample_to_dataset(sample, test)
        result_predictor = model.predict(sample_test_ds)[0][0]

        candidates.append((i, j, en_scaled, result_predictor))
        
        # print (ijd, thisclose, lastclose, cd, en_scaled, result_predictor)

    candidates.sort(key=lambda x: -x[3])
    # candidates.sort(key=lambda x: x[2])

    # chosen move
    i = candidates[0][0]
    j = candidates[0][1]



    lasts = s
    lasti = abs(i)
    lastj = abs(j)
    if i > 0:
        s = s[:i-1] + "(" + s[i:j-1] + ")" + s[j:]
    if i < 0:
        s = s[:-i-1] + "." + s[-i:-j-1] + "." + s[-j:]

    en_new = fc.eval_structure(s)
    if en_new > en_max:
        en_max = en_new

    # print (s, i, j, en_new, en_max)

    found_moves.append((i, j))

    if s==s2:
        break



In [195]:
print_moves(sequence, s1, s2, found_moves)

GGCUCAGUCGACGCUGCACCGCCUAGACCCGAUCGCUAAGAAGGAAAACAGUCAGAUGACCGAGUAUUCGAACGCUGCCGUAUCUCUCGCAUAUAACACA
(((.((((....))))....))).....((..((.....)).))......(((....)))((((....((........)).....))))........... [   0,    0 ] -12.90
(((.((((....))))....))).....((..((.....)).))......(((....)))(((.....((........))......)))........... [ -64,  -86 ]  -9.30
(((.((((....))))....))).....((..((.....)).))......(((....)))((......((........)).......))........... [ -63,  -87 ]  -7.40
(((.((((....))))....))).....((..((.....)).))......(((....)))(.......((........))........)........... [ -62,  -88 ]  -4.90
(((.((((....))))....))).....((..((.....)).))......(((....)))........((........)).................... [ -61,  -89 ]  -7.80
(((.((((....))))....))).....((..((.....)).))......(((....))).........(........)..................... [ -69,  -80 ]  -6.20
(((.((((....))))....))).....((..((.....)).))......(((....)))........................................ [ -70,  -79 ]  -8.70
(((.((((....))))....))).....((..((.....)).)).

-1.4